# Data Discovery: Collect HLS links with specifc Bands
All instructions are running on crycloud. See README file for setting up enviornment and runnig at local computer.

---

### Import Required Packages

In [ ]:
import requests
from pystac_client import Client       # https://pystac-client.readthedocs.io/en/latest/index.html  
from collections import defaultdict    
import json
import geopandas
import geoviews as gv
from cartopy import crs
gv.extension('bokeh', 'matplotlib')

### Submit `GET` request to the CMR STAC API

Use the `reqests` package to submit a `GET` request to the CMR STAC API. We'll parse the response and extract the information we need to navigate the STAC Catalog.

In [ ]:
stac_url = 'https://cmr.earthdata.nasa.gov/stac'

In [ ]:
provider_cat = requests.get(stac_url)

The CMR STAC API endpoint lists the available providers. Each **provider** is a seperate STAC Catalog endpoint that can be used to submit spatiotemporal queries agaist.

In [ ]:
providers = [p['title'] for p in provider_cat.json()['links'] if 'child' in p['rel']]
providers

### Explore the `LPCLOUD` provider

In [ ]:
provider = 'LPCLOUD'

In [ ]:
provider_url = f'{stac_url}/{provider}'
provider_url

In [ ]:
cat = requests.get(provider_url)
cat.json()

### List the STAC Collections within the `LPCLOUD` Catalog

In [ ]:
cols = [{l['href'].split('/')[-1]: l['href']} for l in cat.json() ['links'] if 'child' in l['rel']]
for c in cols:
    print(c)

In [ ]:
try:
    print(f"Requesting page:")
    while nxt_pg := [l for l in cat.json()['links'] if 'next' in l['rel']][0]:
        print(f"{nxt_pg['href'].split('=')[-1]}...", end = ' ')
        cat = requests.get(nxt_pg['href'])
        cols.extend([{l['href'].split('/')[-1]: l['href']} for l in cat.json()['links']if 'child' in l['rel']])
except:
    print('No additional pages')

### Print all available STAC Collections within the `LPCLOUD` Catalog

In [ ]:
print(f'LPCLOUD has {len(cols)} Collections')

In [ ]:
for c in cols:
    print(c)

### Get information about an individual Collection

Below we'll specify a STAC Collection, `HLSL30.v2.0`, and request the STAC metadata

In [ ]:
collection = 'HLSL30.v2.0'

In [ ]:
collection_link = list(filter(lambda c: collection == list(c.keys())[0], cols))[0]
collection_link

In [ ]:
requests.get(collection_link[collection]).json()

### Set up query parameters to submit to the CMR-STAC API

For this next sections we'll use a Python package call `pystac_client` to submit a spatiotemporal querie for data assets across multiple Collections. We will define our area of interest using the geojson file from the previous exercise, while also specifying the data collections and time range of needed for our example.

In [ ]:
field = geopandas.read_file('../data/ne_w_agfields.geojson')
field

In [ ]:
fieldShape = field['geometry'][0]
fieldShape

In [ ]:
base = gv.tile_sources.EsriImagery.opts(width=650, height=500)
farmField = gv.Polygons(fieldShape).opts(line_color='yellow', line_width=10, color=None)
base * farmField

We will now start to specify the search criteria we are interested in, i.e, the **date range**, the **region of interest** (roi), and the **data collections**, to pass to the STAC API

#### Specify the region of interest

In [ ]:
roi = json.loads(field.to_json())['features'][0]['geometry']
print(roi)
roi_pt = json.loads('{"type":"Point", "coordinates":[-119.256, 37.901]}') # Dana meadow
print(roi_pt)

#### Specify date range

In [ ]:
date_range = "2019-07/2019-08"

#### Specify the STAC Collections

**Note,** a STAC Collection is synonomous with what we usually consider a data product.

In [ ]:
collections = ['HLSL30.v2.0', 'HLSS30.v2.0']
collections

### Perform Search Against the CMR-STAC API

In [ ]:
catalog = Client.open(provider_url)

In [ ]:
search = catalog.search(
    collections=collections,
    intersects=roi_pt,
    datetime=date_range
)

#### Print out how many STAC Items match our search query

In [ ]:
search.matched()

We now have a search object containing the STAC records that matched our query. Now, let's pull out all of the STAC Items (as a PySTAC ItemCollection object) and explore the contents (i.e., the STAC Items)

In [ ]:
item_collection = list(search.items())

In [ ]:
item_collection

#### Grab the first Item and print it out as a dictionary

In [ ]:
item_collection[2].to_dict()

### Filtering STAC Items

Now we will set the max cloud cover allowable and extract the band links for those Items that match or are less than the max cloud cover.

In [ ]:
cloudcover = 20

We will also specify the STAC Assets (i.e., bands/layers) of interest for both the S30 and L30 collections

In [ ]:
s30_bands = ['B02', 'B04']
l30_bands = ['B02', 'B04']

In [ ]:
evi_band_links = []

for i in item_collection:
    if i.properties['eo:cloud_cover'] <= cloudcover:
        if i.collection_id == 'HLSS30.v2.0':
            #print(i.properties['eo:cloud_cover'])
            evi_bands = s30_bands
        elif i.collection_id == 'HLSL30.v2.0':
            #print(i.properties['eo:cloud_cover'])
            evi_bands = l30_bands

        for a in i.assets:
            if any(b==a for b in evi_bands):
                evi_band_links.append(i.assets[a].href)

In [ ]:
len(evi_band_links)/4    # Print the number of Items that match our cloud criteria 

### Split Data Links List into Logical Groupings
Open Sentinel Hub.
Select “Harmonized Landsat Sentinel” checkbox on the left, change the cloud cover slider to < 20%.

 Then you can find a region you want with different UTM tile.
eg. s3://lp-prod-protected/HLSS30.020/HLS.S30.T10SEF.2024102T185919.v2.0, where UTM tile is 'T10SEF'and you want to split the links by the tiles you want.


Split by Universal Transverse Mercator (UTM) tile specified in the file name (e.g., T11SLB)

In [ ]:
tile_dicts = defaultdict(list)    # https://stackoverflow.com/questions/26367812/appending-to-list-in-python-dictionary

In [ ]:
for l in evi_band_links:
    tile = l.split('.')[-6]
    tile_dicts[tile].append(l)

#### Print dictionary keys and values, i.e. the data links

In [ ]:
tile_dicts.keys()

In [ ]:
tile_dicts['T11SLB'][:5]

Now we will create a seperate list of data links for each tile

In [ ]:
tile_links_T11SLC = tile_dicts['T11SLC']
tile_links_T11SKC = tile_dicts['T11SKC']
tile_links_T11SKB = tile_dicts['T11SKB']
tile_links_T11SLB = tile_dicts['T11SLB']

#### Print band/layer links for HLS tile T11SLB

In [ ]:
tile_links_T11SLB[:10]

#### Split the links by band

In [ ]:
bands_dicts = defaultdict(list)

In [ ]:
for b in tile_links_T11SLB:
    band = b.split('.')[-2]
    bands_dicts[band].append(b)

In [ ]:
bands_dicts.keys()

### Save links to a text file


#### Write links from CMR-STAC API to a file
We will generate two types of links: HTTPS and S3. HTTPS is a direct link to the NASA Earthdata data library, which you can input into your browser and download the .tif files manually. S3 links allow us to directly access Earthdata in code, avoiding the need to save the .tif files to disk.

In [ ]:
print(bands_dicts.keys())

In [ ]:
for k, v in bands_dicts.items():
    name = (f'HTTPS_T11SLB_{k}_Links.txt')
    with open(f'../data/{name}', 'w') as f:
        for l in v:
            f.write(f"{l}" + '\n')

#### Write links to file for S3 access

In [ ]:
for k, v in bands_dicts.items():
    name = (f'S3_T11SLB_{k}_Links.txt')
    with open(f'../data/{name}', 'w') as f:
        for l in v:
            s3l = l.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://')
            f.write(f"{s3l}" + '\n')